# Two-mode squeezed state model

Use the multi-head (2-head) gates in the phase space 
to create a network that represent a two mode squeezed vacuum
by starting from a Gaussian state and making a pullback

The state propagate in a complex medium

<img src="../img/logo_circular.png" width="20" height="20" />@by claudio<br>


nonlinearxwaves@gmail.com<br>
@created 10 december 2020<br>
@version 23 sep 2023

In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' # disable warning messages

In [2]:
import numpy as np
from thqml import phasespace as ps
from thqml.utilities import utilities
import tensorflow as tf
import tensorflow.keras as tfk
from tensorflow import keras
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping

In [3]:
tf_complex = tf.complex
tf_real = tf.float32
np_complex = complex
np_real = np.float64

In [4]:
tf.keras.backend.clear_session()

In [5]:
np.set_printoptions(precision=2)

## Dimension (number of modes times 2)

In [6]:
N = 8

## Index of the squeezed mode 1 (between 0 an N/2)

In [7]:
n_s1=1

## Index of the squeezed mode 2 

In [8]:
n_s2=3

## Displacer

In [9]:
dtarget=3.2*np.ones((N,1))
norm = np.linalg.norm(dtarget)
dtarget = dtarget /norm
displacer = ps.DisplacementLayerConstant(dtarget)

## Build vacuum by the Gaussian state

In [10]:
vacuum = ps.VacuumLayer(N)

## Two mode squeezer

In [11]:
r_np=1.0;
theta_np=np.pi/7;

In [12]:
squeezer2m=ps.TwoModeSqueezerLayer(N, r_np=r_np, theta_np=theta_np, n_s1=n_s1, n_s2=n_s2)

In [13]:
Ms, MsI=squeezer2m.get_M(); Ms_np = Ms.numpy();print(Ms_np)

[[ 1.    0.    0.    0.    0.    0.    0.    0.  ]
 [ 0.    1.    0.    0.    0.    0.    0.    0.  ]
 [ 0.    0.    1.54  0.    0.    0.   -1.06 -0.51]
 [ 0.    0.    0.    1.54  0.    0.   -0.51  1.06]
 [ 0.    0.    0.    0.    1.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    1.    0.    0.  ]
 [ 0.    0.   -1.06 -0.51  0.    0.    1.54  0.  ]
 [ 0.    0.   -0.51  1.06  0.    0.    0.    1.54]]


### Check if the matrix is symplectic

In [14]:
utilities.printonscreennp(np.matmul(Ms.numpy(),MsI.numpy()))

+1.0+0.0i +0.0+0.0i +0.0+0.0i +0.0+0.0i +0.0+0.0i +0.0+0.0i +0.0+0.0i +0.0+0.0i 
+0.0+0.0i +1.0+0.0i +0.0+0.0i +0.0+0.0i +0.0+0.0i +0.0+0.0i +0.0+0.0i +0.0+0.0i 
+0.0+0.0i +0.0+0.0i +1.0+0.0i +0.0+0.0i +0.0+0.0i +0.0+0.0i -0.0+0.0i +0.0+0.0i 
+0.0+0.0i +0.0+0.0i +0.0+0.0i +1.0+0.0i +0.0+0.0i +0.0+0.0i +0.0+0.0i +0.0+0.0i 
+0.0+0.0i +0.0+0.0i +0.0+0.0i +0.0+0.0i +1.0+0.0i +0.0+0.0i +0.0+0.0i +0.0+0.0i 
+0.0+0.0i +0.0+0.0i +0.0+0.0i +0.0+0.0i +0.0+0.0i +1.0+0.0i +0.0+0.0i +0.0+0.0i 
+0.0+0.0i +0.0+0.0i +0.0+0.0i +0.0+0.0i +0.0+0.0i +0.0+0.0i +1.0+0.0i +0.0+0.0i 
+0.0+0.0i +0.0+0.0i +0.0+0.0i -0.0+0.0i +0.0+0.0i +0.0+0.0i +0.0+0.0i +1.0+0.0i 


The previous matrix must be unitary

# Build the model

In [15]:
xin = tf.keras.layers.Input(N)
x1, d1 = squeezer2m(xin)
chir, chii = vacuum(x1, d1)
squeezed2m = tf.keras.Model(inputs = xin, outputs=[chir, chii])

# Evaluate the covariance matrix

In [16]:
cov_layer = ps.CovarianceLayer(N)
covariance_matrix, mean_R,_ = cov_layer(chir,chii, squeezed2m)
squeezed2m_cov = tf.keras.Model(inputs = xin, outputs=[covariance_matrix, mean_R])

# Training points

In [17]:
xtrain = np.random.rand(1, N)-0.5

# Covariance and displacement

In [18]:
cov0,d0=squeezed2m_cov(xtrain); 

## Displacement

In [19]:
print(d0)

tf.Tensor([[0. 0. 0. 0. 0. 0. 0. 0.]], shape=(1, 8), dtype=float32)


## Eigenvalues and eigenvectors of the covariance matrix

In [20]:
np.linalg.eig(cov0.numpy())

(array([7.39, 0.14, 0.14, 7.39, 1.  , 1.  , 1.  , 1.  ], dtype=float32),
 array([[ 0.  ,  0.  ,  0.  ,  0.  ,  1.  ,  0.  ,  0.  ,  0.  ],
        [ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  1.  ,  0.  ,  0.  ],
        [-0.64, -0.64, -0.23,  0.16,  0.  ,  0.  ,  0.  ,  0.  ],
        [-0.31, -0.31,  0.67, -0.69,  0.  ,  0.  ,  0.  ,  0.  ],
        [ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  1.  ,  0.  ],
        [ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  1.  ],
        [ 0.71, -0.71,  0.08,  0.15,  0.  ,  0.  ,  0.  ,  0.  ],
        [ 0.  ,  0.  , -0.7 , -0.69,  0.  ,  0.  ,  0.  ,  0.  ]],
       dtype=float32))

Note that the squeezed eigenvectors (with not unitary eigenvalues) spans the space of modes n_s1 and n_s2